In [3]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from itertools import permutations
import functools

In [4]:
df = pd.read_csv('virus_genome/human_infecting_virus', delimiter='\t', header=None)
df[['ID', 'DNA Sequence']] = df[0].str.split(expand=True)
df = df.drop(0, axis=1)
df['isZoonotic'] = 1


df2 = pd.read_csv('virus_genome/Other_viruses', delimiter='\t', header=None)
df2[['ID', 'DNA Sequence']] = df2[0].str.split(expand=True)
df2 = df2.drop(0, axis=1)
df2['isZoonotic'] = 0

dfcopy = df2.copy()

dfother = pd.DataFrame()
it = len(df)+1
kmer = 4

# iterate through each sequence
for i in range(len(df2)):
    dfother.at[it, 'DNA Sequence']=df2.at[i, 'DNA Sequence']
    dfother.at[it, 'isZoonotic']=int(0)
    dfother.at[it, 'ID']=df2.at[i, 'ID']

    it+=1


mergedDf = pd.concat([df, dfother], axis=0)


In [ ]:
print(df.shape)
print(df2.shape)

print(mergedDf)

(1236, 3)
(8192, 3)
           ID                                       DNA Sequence  isZoonotic
0       37121  cttttaacaatcatagttttataaaagggtgtaaccgaagcggttt...         1.0
1      129724  atactttacaattatcttgtaaaaagtagggtgtaaccgaaaaggg...         1.0
2       59303  atggcggacgtgtgacatcaccgttcgctctttctaggatcctttg...         1.0
3      120686  ttgttctacttcttactcattattataaattataatgtttgtataa...         1.0
4       99565  gtgaatgatgatggcgtcgaaagacgtcgttgcagctactgctgct...         1.0
...       ...                                                ...         ...
9424   268315  ggatccacagaactccttgtatgtacagccgcgggtacccacagca...         0.0
9425   130329  ctcatgtcattaataagaccatgcagaaaatgcaaatgaggcgaag...         0.0
9426  1239437  ccgaagtgggatggcggctgccgccgcttcggcgcttggcgcttcg...         0.0
9427   358812  gtatacgaggttagttcattctcgtatacacgattggacaaatcaa...         0.0
9428    11287  atggaaggaggaattcgtgcagcgttttcaggcctgaatgatgtta...         0.0

[9428 rows x 3 columns]


In [ ]:
# generate permutations of a kmer value
kmer = 4
l = list(permutations('acgt'))
print(len(l))

24


In [ ]:
dna = mergedDf['DNA Sequence']
Y = mergedDf['isZoonotic']
print(mergedDf.loc[0])
kmer = 4
for i in range(2): # tweak numbers
    for j in range(20):
        st = str(dna.iloc[i])
        print(st[j:j+kmer])
        try:
            mergedDf.at[i,st[j:j+kmer]]+=1
        except Exception as e:
            mergedDf.at[i,st[j:j+kmer]]= 1




ID                                                          37121
DNA Sequence    cttttaacaatcatagttttataaaagggtgtaaccgaagcggttt...
isZoonotic                                                    1.0
Name: 0, dtype: object
cttt
tttt
ttta
ttaa
taac
aaca
acaa
caat
aatc
atca
tcat
cata
atag
tagt
agtt
gttt
tttt
ttta
ttat
tata
atac
tact
actt
cttt
ttta
ttac
taca
acaa
caat
aatt
atta
ttat
tatc
atct
tctt
cttg
ttgt
tgta
gtaa
taaa


In [ ]:
mergedDf.fillna(0, inplace=True)

X = mergedDf[mergedDf.columns[4:len(mergedDf.columns)]]
Y = mergedDf['isZoonotic']



print(X)
print(Y)


      tttt  ttta  ttaa  taac  aaca  acaa  caat  aatc  atca  tcat  ...  aatt  \
0      2.0   2.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   0.0   
1      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   1.0   
2      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
3      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
9424   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
9425   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
9426   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
9427   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
9428   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

      atta  tatc  atct  tctt  cttg  ttgt  tgta  gta

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=1)

print(X_test)


      tttt  ttta  ttaa  taac  aaca  acaa  caat  aatc  atca  tcat  ...  aatt  \
1303   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
6597   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4096   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
7285   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4900   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
4287   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
5019   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
422    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
7322   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
5932   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

      atta  tatc  atct  tctt  cttg  ttgt  tgta  gta

In [ ]:
model = LogisticRegression(
    multi_class="multinomial", max_iter=1000,
    fit_intercept=False, tol=0.001, solver='saga', random_state=42)

In [ ]:
model.fit(X_train, y_train)

LogisticRegression(fit_intercept=False, max_iter=1000,
                   multi_class='multinomial', random_state=42, solver='saga',
                   tol=0.001)

In [ ]:
predictions = model.predict(X_test)
print(X_test)

print(model.score(X_test, predictions))


      tttt  ttta  ttaa  taac  aaca  acaa  caat  aatc  atca  tcat  ...  aatt  \
1303   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
6597   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4096   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
7285   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4900   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
4287   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
5019   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
422    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
7322   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
5932   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

      atta  tatc  atct  tctt  cttg  ttgt  tgta  gta

In [ ]:
print(model.predict())

a bcdef
